In [1]:
import json
import os
from datetime import datetime
import pandas as pd
import random
import py7zr
import glob
import re
import pickle as pkl

In [ ]:
def ultimate_filter(unfilteredInputfile_path, outpath):    
    
    page_object=[] #wird eine Liste von dicts
    result=list() #Sammelliste für alle Tabellen mit min. 2 row change
       with open(unfilteredInputfile_path, 'r', encoding="utf8") as f:
        data_prev = {}
        current=json.loads(next(f))
        for line in f:

            data=json.loads(line)
            if data_prev =={} : #Beim allerersten Durchgang
               # print("Aller Anfang")
                page_object = []
                page_object.append(current) #beim ersten Durchgang muss das mit dem current dict erstellt werden
                # alles mit der gleichen pageID in ein Objekt zusammenpacken
            elif current["pageID"] != data_prev["pageID"] and current["pageID"]==data["pageID"]:
                #print("NEW PAGE IS BEGINNING")
                #empty the page object and start over to fill it
                page_object = []
                page_object.append(current)

            elif current["pageID"] == data_prev["pageID"] and current["pageID"]==data["pageID"]:#wenn die Seite gleichbleibt
                #print("same old")
                page_object.append(data)

            elif current["pageID"] == data_prev["pageID"] and current["pageID"]!=data["pageID"]: #Seite wird sich ändern
                #object for this page is now complete
                #print("Page is about to change")
                #filter tables within that page
                #filter for list/dict per table
                #sort collected tables by key
                page_object=sorted(page_object, key=lambda d: d['key'])
                key_prev=page_object[0]["key"]#am Anfang ist der vorherige Key gleich dem aktuellen der Einfachheit halber
                tab_object = []
                for tab in range(len(page_object)):

                    if page_object[tab]["key"]== key_prev:
                        tab_object.append(page_object[tab])
                        #print("Type: ", page_object[tab]["contentType"], "tab0")
                    else:
                        #before creating a new tab_object, process the current one
                        #alle dicts, die keine rows haben, bekommen 0 zugewiesen
                        for i in range(len(tab_object)):
                            tab_object[i]["rows"]=tab_object[i].get("rows",0)
                        #sort by key 
                        seq=[x['rows'] for x in tab_object]
                        
                        
                        if len(tab_object)>2 and max(seq)>2 and tab_object[len(tab_object)-1]["contentType"]!="DELETE":#Tabellen, die nur einmal auftreten ausschließen und solche mit zwei oder weniger Zeilen
                            if len(tab_object)<3:
                                print(len(tab_object), "ZU WENIGE ENTRIES!!!!!!!!", tab_object[len(tab_object)-1]["key"])
                            #print("Number of rows in table: ")
                            if max(seq)<3:
                                print(max(seq), "F...., die rows")
                            
                            if tab_object[len(tab_object)-1]["contentType"]=="DELETE":
                                print(tab_object[len(tab_object)-1]["key"], "SHIT")
                            
                            
                            #filter within table objects:remove if too short distance between edits or UNMODIFIED
                            tab_object=sorted(tab_object, key=lambda d: d['validFrom'])
                            filtered_versions=[]
                            prev_version = tab_object.index(tab_object[0])  # am Anfang ist der vorherige Key gleich dem aktuellen der Einfachheit halber
                            for version in range(len(tab_object)):
                                #distance too short, remove
                                if (abs(datetime.strptime(tab_object[prev_version]["validFrom"], '%Y-%m-%dT%H:%M:%SZ') - datetime.strptime(tab_object[version]["validFrom"], '%Y-%m-%dT%H:%M:%SZ')).days > 0\
                                        and tab_object[version]["contentType"]!="UNMODIFIED" and tab_object[version]["contentType"]!="DELETE" ) or tab_object[version]["contentType"]=="CREATE":
                                    # or CREATE  is important to keepthe create element of that table (comparsion in the first iteration with itself)
                                    #print("Nice Version detected")
                                    filtered_versions.append(tab_object[version])
                                    prev_version=tab_object.index(tab_object[version])

                            filtered_versions = sorted(filtered_versions, key=lambda d: (d['key'],d['validFrom']))
                            if len(filtered_versions)>2 and filtered_versions[len(filtered_versions)-1]["rows"]>0:#make sure after all that filtering still enough entries remain for that table
                                if filtered_versions[len(filtered_versions)-1]["rows"]<1:
                                    print("LastRows: ",filtered_versions[len(filtered_versions)-1]["rows"])
                                #and the table should have more than 0 rows
                                
                                filename=outpath + tab_object[0]['key'] +'.json'
                                #print(filename)
                               
                                with open(filename, 'w') as fp:
                                    json.dump(filtered_versions, fp)#tab_object

                            
                        #print("new tab object created")
                        tab_object=[]
                        tab_object.append(page_object[tab])
                    key_prev=page_object[tab]["key"]
                    #print(key_prev)



            else: # if nothig of the above is applicable
                #print("ELSE")
                page_object={}
                page_object.update(data)
            #reset prev, current and next
            data_prev=current
            current=data

In [ ]:
#merge all json files into one
def collect_filtered_files(inpath, pagefilename ):
    result=list() #Sammelliste für alle Tabellen mit min. 2 row change
    jsonfiles=[f for f in os.listdir(inpath) if f.endswith('.json')]
    os.chdir(inpath)
    for f1 in jsonfiles:
        with open(f1, 'r') as infile:
            result.extend(json.load(infile))
    with open(pagefilename, 'w') as output_file:
        json.dump(result, output_file)

In [ ]:
def read_filtered_json(json_inpath):    

    with open(json_inpath) as json_file:
        data=json.load(json_file)

    #convert the json-file to a dataframe
    tabledf=pd.DataFrame.from_dict(data, orient="columns")
    return tabledf

In [ ]:
def preprocess_page(tabledf_page):
    #drop unneccessary columns
    tabledf_page=tabledf_page.drop(columns=['pageID', 'contentHash', 'revisionId','similarityLast',  'comment', 'position','user', 'caption'])
    
    #ValidFrom with datetime format
    tabledf_page["validFrom"]=pd.to_datetime(tabledf_page["validFrom"], format='%Y-%m-%dT%H:%M:%SZ')
    tabledf_page["timestamp_c"]=tabledf_page.validFrom.transform(lambda x: x-x.min())
    
    #generate seperate df for each table on that page
    grouped=tabledf_page.groupby("key")
    result=[g[1]for g in list(grouped)[:]]
    
    return result

In [7]:
### get the files in the order displayed in the explorer ###
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

Step 1:
Download matched tables from https://owncloud.hpi.de/s/oLQ7zrblMJHvNqd and save the files in a directory called "matched-tables"

In [10]:
###rename table files: prepend number (for the selection procedure later on)
path="your-absolute-path-to-matched-tables"
files = sorted_alphanumeric(os.listdir(path))

for index, file in enumerate(files):
    os.rename(os.path.join(path, file), os.path.join(path, ''.join([str(index),"_",file])))

In [11]:
### Create array with random numbers to select random buckets###
random.seed(12)
random_buckets=random.sample(range(656), 25)
random_buckets=sorted(random_buckets)

In [111]:
def create_sample(begin, end, random_buckets, files, path, pklname, tabprobucket):  
    rs_list=[]
    rs_listlist=[]
    ### extract 7z file ###
    os.chdir(path)
    extract_path=path
    os.chdir("..")
    outpath_zip=(os.path.join(os.path.abspath(os.curdir), str("output-table")))
    outpath_filter=(os.path.join(os.path.abspath(os.curdir), str("out_filter")))
    for index in range(begin,end):
        print(random_buckets[index])
        collected_name="result_bucket_"+str(random_buckets[index])+".json"
        if os.path.exists(os.path.join(outpath_filter, str(random_buckets[index]), collected_name)):
            out_filterb=(os.path.join(outpath_filter, str(random_buckets[index])+"\ "))[:-1]
            print(random_buckets[index], ": exists")
        else:    


            #unzip bucket
            print("unzipping...")
            with py7zr.SevenZipFile(os.path.join(path, files[random_buckets[index]]), mode='r') as z:
                z.extractall(path=outpath_zip)

            name=re.sub(r'^.*?enwiki', 'enwiki',os.path.splitext(files[random_buckets[index]])[0])
            print(name)
           
            ##create directory to collect single tables from filtering
            if not os.path.exists(os.path.join(outpath_filter, str(random_buckets[index]))):
                os.mkdir(os.path.join((outpath_filter), str(random_buckets[index])+"\ "))
                out_filterb=os.path.join(outpath_filter, str(random_buckets[index])+"\ ")
            else:
                out_filterb=os.path.join(outpath_filter, str(random_buckets[index])+"\ ")

            #filter bucket   
            print("filter bucket...")
            ultimate_filter(os.path.join(outpath_zip,"output-table", name ), out_filterb) #os.path.join(outpath_zip,"output-table", name )

            #collect filtered tables
            out_filterb=out_filterb[:-1]
            collect_filtered_files(out_filterb, collected_name )

        print("read_filtered...")
        #read data and preprocess
        tabledf=read_filtered_json(os.path.abspath(os.path.join(out_filterb, collected_name)))
        result_samples=preprocess_page(tabledf)

        #Array to randomly select tables
        random.seed(random_buckets[index])
        random_tables=random.sample(range(len(result_samples)),tabprobucket)

        print("extract tables...")
        #list of df containing random tables of that bucket
        rt_list=[result_samples[i] for i in random_tables]

        #collect selected tables
        rs_listlist.append(rt_list)
        rs_list.extend(rt_list)

        print("cleaning up...")
        file_path =os.path.join(out_filterb, collected_name)


        if os.path.isfile(file_path):
            # Verifies CSV file was created, then deletes unneeded files.
            for clean_up in glob.glob(os.path.join(out_filterb,"*.*")):
                #print(clean_up)
                if not clean_up.startswith(os.path.join(out_filterb, collected_name)):#r"C:\Users\johan\Documents\Studium\Masterarbeit\Code\out_filter\11\result_bucket"):    
                    os.remove(clean_up)



        pkl.dump(rs_list, open(str(pklname)+".pkl", "wb"))
        print("done.")

In [113]:
#### generate new random sample for classifier evaluation 
#use tabprobucket=10 to recreate the original classifier sample
create_sample(  0, 25,random_buckets, files, path, "classifier_sample0" , 10)

If memory is not sufficient to generate the sample in one go, use the parameters begin and end of the create_sample function to iteratively create the sample.
Use the cell below to combine the segments of the sample.

In [9]:
ds_all=[]
for i in range(0,5):
    ds=pkl.load(open(r".\classifier_sample"+str(i)+".pkl", "rb"))
    ds_all.extend(ds)
pkl.dump(ds_all, open(r".\classifier_sample_complete.pkl", "wb"))